In [1]:
# ========================
# 📦 IMPORTAÇÕES GERAIS
# ========================
import os
import re
import importlib.util

# ============================
# 📄 MANIPULAÇÃO DE DOCUMENTOS
# ============================
from docx import Document


In [ ]:
# Caminhos base dos arquivos a tratar
ARQUIVO_ORIGINAL = r"F:\OneDrive\Documentos\Github\ScrapStream\tools\Divine Emperor of Death 1801-1800_revisado.docx"
ARQUIVO_PATTERN = os.path.abspath(os.path.join("..", "pattern.py"))


In [3]:
def remover_capitulos_duplicados(caminho_entrada, caminho_saida, caminho_log="log_duplicados.txt"):
    doc = Document(caminho_entrada)
    novo_doc = Document()

    titulos_vistos = set()
    capitulos_duplicados = []
    pular_capitulo = False

    for par in doc.paragraphs:
        if par.style and hasattr(par.style, "name") and par.style.name == "Heading 2":
            titulo = par.text.strip()
            if titulo in titulos_vistos:
                capitulos_duplicados.append(titulo)
                print(f"⚠️ Capítulo duplicado detectado: {titulo}")
                pular_capitulo = True
                continue
            else:
                titulos_vistos.add(titulo)
                novo_doc.add_paragraph(titulo, style="Heading 2")
                pular_capitulo = False
        else:
            if not pular_capitulo:
                novo_doc.add_paragraph(par.text, style=par.style)

    novo_doc.save(caminho_saida)
    print(f"✅ Novo arquivo salvo como: {caminho_saida}")

    if capitulos_duplicados:
        with open(caminho_log, "w", encoding="utf-8") as f:
            f.write("Capítulos duplicados removidos:\n\n")
            for titulo in capitulos_duplicados:
                f.write(f"{titulo}\n")
        print(f"📄 Log salvo como: {caminho_log}")
    else:
        print("✅ Nenhum capítulo duplicado encontrado.")
        
    return capitulos_duplicados  # retorna lista de duplicados


In [4]:
def desfazer_censura(texto):
    corrigidas = []

    def juntar_letras(match):
        original = match.group(0)
        corrigido = original.replace('.', '')
        if corrigido != original:
            corrigidas.append(original)
        return corrigido

    texto_corrigido = re.sub(r'\b(?:[a-zA-Z]\.?){3,}\b', juntar_letras, texto)
    return texto_corrigido, corrigidas

def limpar_censura_docx(caminho_docx):
    doc = Document(caminho_docx)
    novo_doc = Document()
    log_corrigidas = []

    for par in doc.paragraphs:
        texto_corrigido, palavras = desfazer_censura(par.text)
        log_corrigidas.extend(palavras)
        novo_par = novo_doc.add_paragraph(texto_corrigido)
        novo_par.style = par.style

    nome_base, ext = os.path.splitext(caminho_docx)
    caminho_saida = f"{nome_base}_uncens{ext}"
    novo_doc.save(caminho_saida)

    if log_corrigidas:
        caminho_log = f"{nome_base}_uncens_log.txt"
        with open(caminho_log, "w", encoding="utf-8") as f:
            f.write("Palavras censuradas corrigidas:\n\n")
            for palavra in sorted(set(log_corrigidas)):
                f.write(f"{palavra} → {palavra.replace('.', '')}\n")
        print(f"📝 Log salvo em: {caminho_log}")

    return caminho_saida, log_corrigidas

In [5]:
def replace_or_remove_terms(pattern_path, docx_path, replace_with=None):
    spec = importlib.util.spec_from_file_location("pattern_module", pattern_path)
    if spec is None or spec.loader is None:
        raise ImportError(f"Erro ao carregar módulo: {pattern_path}")
    pattern_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(pattern_module)

    terms_to_find = getattr(pattern_module, "pattern", [])
    doc = Document(docx_path)

    ocorrencias = []

    for paragraph in doc.paragraphs:
        for term in terms_to_find:
            if term in paragraph.text:
                paragraph.text = paragraph.text.replace(term, "" if replace_with is None else replace_with)
                ocorrencias.append(term)

    doc.save(docx_path)
    return ocorrencias



In [9]:
def limpar_texto_paragrafo(texto: str) -> str:
    texto = texto.strip()
    texto = re.sub(r"(?im)^\s*the\s+end\.?\s*$", "", texto)
    texto = re.sub(r"(?i)^ *(corrected|edited|fixes|changes|modifications):.*$", "", texto, flags=re.MULTILINE)
    texto = re.sub(r"(?i)^ *[-–•*]?\s*(removed|deleted|eliminated|cleaned)\s+.*$", "", texto, flags=re.MULTILINE)


    return texto.strip()


def remover_artefatos_docx(caminho_arquivo: str) -> tuple[str, int]:
    doc = Document(caminho_arquivo)
    novo_doc = Document()

    num_afetados = 0
    for par in doc.paragraphs:
        texto_original = par.text.strip()
        texto_limpo = limpar_texto_paragrafo(texto_original)
        if texto_original != texto_limpo:
            num_afetados += 1
        if texto_limpo:
            novo_par = novo_doc.add_paragraph(texto_limpo)
            novo_par.style = par.style


    dir_base, nome = os.path.split(caminho_arquivo)
    nome_base, ext = os.path.splitext(nome)
    novo_nome = f"{nome_base}_te{ext}"
    novo_caminho = os.path.join(dir_base, novo_nome)
    novo_doc.save(novo_caminho)
    return novo_caminho, num_afetados


In [10]:
def executar_limpeza_completa(
    caminho_original: str,
    caminho_pattern_py: str = "pattern.py"
) -> str:
    """
    Executa a limpeza completa de um .docx:
    - Remove censura
    - Remove artefatos LLM
    - Remove marcas d’água (pattern)
    - Remove capítulos duplicados

    Gera apenas um arquivo final com sufixo '_pipe.docx' e um log consolidado.
    Remove todos os arquivos intermediários gerados.
    """
    print(f"\n🚀 Iniciando limpeza de: {caminho_original}\n")
    log = []

    # 1. Desfazer censura
    caminho_1, censuradas = limpar_censura_docx(caminho_original)
    log.append(f"[🔓] Censura desfeita: {len(set(censuradas))} palavras corrigidas.")

    # 2. Limpar artefatos LLM
    caminho_2, num_afetados = remover_artefatos_docx(caminho_1)
    log.append(f"[🧹] Artefatos LLM removidos: {num_afetados} parágrafos limpos.")

    # 3. Remover marcas d'água
    termos_removidos = replace_or_remove_terms(caminho_pattern_py, caminho_2)
    log.append(f"[🚫] Watermarks removidas: {len(set(termos_removidos))} termos.")

    # 4. Remover capítulos duplicados
    caminho_final = os.path.splitext(caminho_original)[0] + "_pipe.docx"
    duplicados = remover_capitulos_duplicados(caminho_2, caminho_final)
    log.append(f"[📑] Capítulos duplicados removidos: {len(duplicados)}.")

    # 5. Limpeza dos intermediários
    for f in [caminho_1, caminho_2]:
        try:
            os.remove(f)
        except Exception as e:
            print(f"[⚠️] Erro ao apagar arquivo intermediário {f}: {e}")

    # 6. Log final
    caminho_log = os.path.splitext(caminho_final)[0] + "_log.txt"
    with open(caminho_log, "w", encoding="utf-8") as f:
        f.write(f"📋 Log de limpeza: {os.path.basename(caminho_original)}\n\n")
        for entrada in log:
            f.write(entrada + "\n")
        if duplicados:
            f.write("\n--- Capítulos Duplicados ---\n")
            for cap in duplicados:
                f.write(f"- {cap}\n")
        if censuradas:
            f.write("\n--- Palavras Censuradas Corrigidas ---\n")
            for palavra in sorted(set(censuradas)):
                f.write(f"{palavra} → {palavra.replace('.', '')}\n")
        if termos_removidos:
            f.write("\n--- Watermarks Removidas ---\n")
            for termo in sorted(set(termos_removidos)):
                f.write(f"- {termo}\n")

    print(f"\n✅ Arquivo final salvo como:\n{caminho_final}")
    print(f"📝 Log salvo como:\n{caminho_log}")
    return caminho_final



In [11]:
executar_limpeza_completa(ARQUIVO_ORIGINAL, ARQUIVO_PATTERN)


🚀 Iniciando limpeza de: F:\OneDrive\Documentos\Github\ScrapStream\tools\Divine Emperor of Death 1701-1800_revisado.docx

✅ Novo arquivo salvo como: F:\OneDrive\Documentos\Github\ScrapStream\tools\Divine Emperor of Death 1701-1800_revisado_pipe.docx
✅ Nenhum capítulo duplicado encontrado.

✅ Arquivo final salvo como:
F:\OneDrive\Documentos\Github\ScrapStream\tools\Divine Emperor of Death 1701-1800_revisado_pipe.docx
📝 Log salvo como:
F:\OneDrive\Documentos\Github\ScrapStream\tools\Divine Emperor of Death 1701-1800_revisado_pipe_log.txt


'F:\\OneDrive\\Documentos\\Github\\ScrapStream\\tools\\Divine Emperor of Death 1701-1800_revisado_pipe.docx'